# 네이버 매장정보 크롤링

### 1.공공데이터_to_네이버존재확인에서 만든 파일을 사용합니다

In [50]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  
import chromedriver_autoinstaller

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
from tqdm import tqdm  

데이터 가져오기

In [51]:
# 데이터 불러오기 (네이버에 매장이 없는 경우와, 쓸데없는 컬럼은 삭제합니다.)
original_res_df = pd.read_excel('1_1_공공데이터_to_네이버존재확인.xlsx')
res_df1 = original_res_df.copy()
res_df1 = res_df1.loc[res_df1['검색여부'] == '가능'].reset_index(drop=True) 
res_df1.drop('Unnamed: 0', axis=1, inplace=True) 

# 컬럼 순서 바꾸기 -> 보기 좋으라고 바꿈
res_df1['검색어'] = res_df1['상세주소'] + " " + res_df1['사업장명']
column_order = ['검색어', '업태구분명', '사업장명', '사업장명_원본', '시도', '지역구', '상세주소', '소재지전체주소', '도로명전체주소', '좌표정보(x)', '좌표정보(y)', '검색여부', '영업상태명']
res_df1 = res_df1[column_order]
res_df1.tail()

,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명
8291,"도곡로 401, 간코",일식,간코,간코 롯데백화점강남점,서울특별시,강남구,"도곡로 401,",서울특별시 강남구 대치동 937 롯데백화점,"서울특별시 강남구 도곡로 401, 롯데백화점 지하1층 (대치동)",204669.543367,443873.621189,가능,영업/정상
8292,"압구정로 108, 오토김밥",분식,오토김밥,오토김밥 신사점,서울특별시,강남구,"압구정로 108,",서울특별시 강남구 신사동 528-3 덕산빌딩,"서울특별시 강남구 압구정로 108, 덕산빌딩 지상1층 10호 (신사동)",201749.456414,446742.828124,가능,영업/정상
8293,"압구정로42길 36, 마들마들",한식,마들마들,마들마들,서울특별시,강남구,"압구정로42길 36,",서울특별시 강남구 신사동 635-14 금광아크존1,"서울특별시 강남구 압구정로42길 36, 지상1층 101호 (신사동, 금광아크존1)",202990.870783,447146.555568,가능,영업/정상
8294,"언주로151길 21, 세바",경양식,세바,세바(Sevva),서울특별시,강남구,"언주로151길 21,",서울특별시 강남구 신사동 629-1,"서울특별시 강남구 언주로151길 21, 지상1층 (신사동)",202731.421127,446609.460134,가능,영업/정상
8295,"논현로161길 37, 구구구",경양식,구구구,구구구 (GOOGOOGOO),서울특별시,강남구,"논현로161길 37,",서울특별시 강남구 신사동 552-5,"서울특별시 강남구 논현로161길 37, 동쪽 지상1층 (신사동)",202153.410454,446748.562578,가능,영업/정상


In [52]:
# 크롤링 세팅
'''
한번에 많은 양을 하게 되면 에러가 나니까, 나누어 크롤링합니다. 
star_num은 시작할 index
num은 한번에 크롤링할 갯수 
end_num은 끝낼 index입니다. 
'''

start_num = 3401 # 할 차례, 여기를 계속 수정해주세요. 
num = 98  # 100개당 15분 걸림
end_num = start_num + num
res_df = res_df1[start_num:end_num] # 맨 마지막 크롤링할떄는 end_num을 지우세요. [start_num:] <-이렇게 
res_df = res_df.reset_index(drop=True)
res_df.head(10)


,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명
0,"삼성로104길 16, 뽕사부",중국식,뽕사부,뽕사부 삼성점,서울특별시,강남구,"삼성로104길 16,",서울특별시 강남구 삼성동 152-54번지,"서울특별시 강남구 삼성로104길 16, 1층 (삼성동)",204891.083559,445417.156401,가능,영업/정상
1,"언주로 118, 카페아리",기타,카페아리,카페아리,서울특별시,강남구,"언주로 118,",서울특별시 강남구 도곡동 467-24번지,"서울특별시 강남구 언주로 118, 1층 123호 (도곡동, 우성캐릭터199)",204518.879980,442703.584824,가능,영업/정상
2,"테헤란로8길 11-4, 육전식당",한식,육전식당,육전식당 4호점,서울특별시,강남구,"테헤란로8길 11-4,",서울특별시 강남구 역삼동 823-10,"서울특별시 강남구 테헤란로8길 11-4, 1층 (역삼동)",202759.330000,443984.855000,가능,영업/정상
3,"테헤란로22길 12, 평방옥",경양식,평방옥,평방옥,서울특별시,강남구,"테헤란로22길 12,",서울특별시 강남구 역삼동 736-32번지,"서울특별시 강남구 테헤란로22길 12, 지상1층 (역삼동)",203023.476824,444097.769813,가능,영업/정상
4,"강남대로152길 67, 최정참치",일식,최정참치,최정참치,서울특별시,강남구,"강남대로152길 67,",서울특별시 강남구 신사동 540-19,"서울특별시 강남구 강남대로152길 67, 지상2층 201호 (신사동)",202098.989146,446311.090743,가능,영업/정상
5,"삼성로147길 4, 스테파노 일류초밥",일식,스테파노 일류초밥,스테파노 일류초밥,서울특별시,강남구,"삼성로147길 4,",서울특별시 강남구 청담동 7-21번지,"서울특별시 강남구 삼성로147길 4, 1층 (청담동)",204142.589347,446723.577115,가능,영업/정상
6,"영동대로85길 9, 대독장",한식,대독장,대독장 삼성역점,서울특별시,강남구,"영동대로85길 9,",서울특별시 강남구 대치동 947번지,"서울특별시 강남구 영동대로85길 9, 지상2층 (대치동)",205492.595000,444987.990000,가능,영업/정상
7,"영동대로138길 15, 우노커피",경양식,우노커피,우노커피,서울특별시,강남구,"영동대로138길 15,",서울특별시 강남구 청담동 131-13번지,"서울특별시 강남구 영동대로138길 15, 1층 101호 (청담동)",204919.074139,446773.705671,가능,영업/정상
8,"광평로51길 8, 메가엠지씨커피",중국식,메가엠지씨커피,메가엠지씨커피 수서중앙점,서울특별시,강남구,"광평로51길 8,",서울특별시 강남구 수서동 714,"서울특별시 강남구 광평로51길 8, 1층 105호 (수서동, 로얄프라자)",208992.862909,442912.827555,가능,영업/정상
9,"선릉로132길 15, 으뜸청담맛집",일식,으뜸청담맛집,으뜸청담맛집,서울특별시,강남구,"선릉로132길 15,",서울특별시 강남구 청담동 38-19,"서울특별시 강남구 선릉로132길 15, 지상1층 (청담동)",203682.437396,446235.879405,가능,영업/정상


In [53]:
options = webdriver.ChromeOptions()
options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화

# 새로운 창이 뜨지 않도록 설정
options.add_argument("--disable-popup-blocking")
options.add_experimental_option("excludeSwitches", ["enable-popup-blocking"])


# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정 
# driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
chromedriver_autoinstaller.install()
driver = webdriver.Chrome(options=options)

크롤링 시작

In [54]:

# 가져올 정보들에 대한 리스트 지정
# 가게명, 업태명, 주소, 영업시간(##지선,지은 추가##)+ 주력메뉴(맨첫번째메뉴) + 가격까지, 방문자리뷰수,블로그리뷰수
restaurant_name_list = []
category_name_list = []
address_list = []
restaurant_time_list =[]
menu_list = []
price_list = []
visitor_review_list =[]
blog_review_list =[]

import time
from selenium.common.exceptions import NoSuchElementException

# 크롤링 시작 
total_iterations = len(res_df) # 크롤링 진행 과정을 파악하기 위해 tqdm 사용 
for i in tqdm(range(total_iterations), desc='진행 중', position=0, leave=True):

    # 검색어를 네이버 url에 포함시켜 이동 + 검색  ex) 일원로9길 6 치어스
    name = res_df['검색어'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))
    time.sleep(5)  

    ## 화면이 안넘어가고 검색결과에 머물러 있을 때는 entryIframe 이 없음
    ## 그래서 검색어 쳤을때 entryIframe 있으면 --> 화면이 넘어갔다는 뜻이고 
    ## 없으면 아직 안넘어간 뜻이니깐 searchIframe 에서 클릭해서 넘어가야함 .
    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)            
            time.sleep(3) 
            

            # 가게 이름, 변수로 지정
            # 메뉴와 가격이 존재하는지의 여부를 고려 
            # 매장마다 메뉴를 보여주는 테마가 2가지 존재하기 때문에 이를 고려
            restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
            category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
            address = driver.find_element(By.CLASS_NAME,'LDgIH').text
            if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                price = driver.find_element(By.CLASS_NAME,'mkBm3').text
            elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                price = driver.find_element(By.CLASS_NAME,'Yrsei').text
            else : 
                menu = '메뉴없음'
                price = "가격없음"

                ###''지선,지은 추가'': 영업시간 버튼 클릭 후 실행##               
            button = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[5]/div/div[2]/div[1]/div/div[3]/div/a')
            driver.execute_script("arguments[0].click();", button)
            business_hours = driver.find_elements(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.pSavy > div > a")
            for hours in business_hours:
                res_hours_list = hours.text.split('\n')
                res_hours = ' '.join(res_hours_list)

            # 리뷰 긁어오기 
            # PXMot 클래스 요소들을 모두 찾음
            # PXMot는 리뷰의 클래스 이름인데 별점, 방문자리뷰, 블로그리뷰 총 3개가 존재
            pxmot_elements = driver.find_elements(By.CLASS_NAME, 'PXMot')            
            visitor_review = 0
            blog_review = 0

            # PXMot 가 존재할 때 (별점,방문자,블로그 정보가 하나라도 존재)
            if pxmot_elements:
                for element in pxmot_elements:
                    element_text = element.text
                    if element_text.startswith('별점'):
                        # 별점이 있으면 다음 요소 검사
                        continue
                    elif element_text.startswith('방문자리뷰'):
                        # 방문자 리뷰가 존재하는 경우
                        review_text = element_text.replace('방문자리뷰', '')
                        # 쉼표(,) 제거 후 정수로 변환
                        try : 
                            visitor_review = int(review_text.replace(',', '')) if ',' in review_text else int(review_text)
                        except ValueError :
                            visitor_review = 0
                    elif element_text.startswith('블로그리뷰'):
                        # 블로그 리뷰가 존재하는 경우
                        review_text = element_text.replace('블로그리뷰', '')
                        # 쉼표(,) 제거 후 정수로 변환
                        try :
                            blog_review = int(review_text.replace(',', '')) if ',' in review_text else int(review_text)
                        except ValueError :
                            blog_review = 0
            # PXMot 가 없을 때(별점 방문자 블로그 중  뭐라도 존재x)
            else  : 
                visitor_review = 0
                blog_review = 0      


        else: 
            # searchIframe 프레임 
            # searchIframe 프레임으로 가야하는 경우는 검색했을 때 결과가 바로 안나오고 식당리스트가 먼저나오는 경우
            # 그래서 searchIFrame에서 첫번쨰 식당을 선택하고 그다음에 entryIFrame으로 들어가야함
            searchIframe = driver.find_element(By.ID,'searchIframe')
            driver.switch_to.frame(searchIframe)

            ## 클릭 
            driver.find_element(By.CLASS_NAME,'YwYLL').click()
            
            # 기본프레임 
            time.sleep(1)
            driver.switch_to.default_content()        

            # entryIframe 프레임 
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
            try : 
                time.sleep(3) 
            except :
                pass


            # 가게 이름, 변수로 지정
            restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
            category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
            address = driver.find_element(By.CLASS_NAME,'LDgIH').text

            ###''지선,지은 추가''##
            # 영업시간 버튼 클릭 후 실행, 텍스트 추출##
            button = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[5]/div/div[2]/div[1]/div/div[3]/div/a')
            driver.execute_script("arguments[0].click();", button)
            business_hours = driver.find_elements(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.pSavy > div > a")
            for hours in business_hours:
                res_hours_list = hours.text.split('\n')
                res_hours = ' '.join(res_hours_list)

            # 메뉴와 가격이 존재하는지의 여부를 고려 
            # 매장마다 메뉴를 보여주는 테마가 2가지 존재하기 때문에 이를 고려
            if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                price = driver.find_element(By.CLASS_NAME,'mkBm3').text
            elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                price = driver.find_element(By.CLASS_NAME,'Yrsei').text
            else : 
                menu = '메뉴없음'
                price = "가격없음"

            # 리뷰 수 긁어오기
            # PXMot 클래스 요소들을 모두 찾음
            # PXMot 는 리뷰의 클래스 이름인데 별점, 방문자리뷰, 블로그리뷰 총 3개가 존재
            pxmot_elements = driver.find_elements(By.CLASS_NAME, 'PXMot')
        
            # PXMot 가 존재할 때 (별점,방문자,블로그 정보가 하나라도 존재)
            if pxmot_elements:
                for element in pxmot_elements:
                    element_text = element.text
                    if element_text.startswith('별점'):
                        # 별점이 있으면 다음 요소 검사
                        continue
                    elif element_text.startswith('방문자리뷰'):
                        # 방문자 리뷰가 존재하는 경우
                        review_text = element_text.replace('방문자리뷰', '')
                        # 쉼표(,) 제거 후 정수로 변환
                        try : 
                            visitor_review = int(review_text.replace(',', '')) if ',' in review_text else int(review_text)
                        except ValueError :
                            visitor_review = 0
                    elif element_text.startswith('블로그리뷰'):
                        # 블로그 리뷰가 존재하는 경우
                        review_text = element_text.replace('블로그리뷰', '').strip()
                        # 쉼표(,) 제거 후 정수로 변환
                        try :
                            blog_review = int(review_text.replace(',', '')) if ',' in review_text else int(review_text)
                        except ValueError :
                            blog_review = 0
            # PXMot 가 없을 때(별점 방문자 블로그 중  뭐라도 존재x)
            else  : 
                visitor_review = 0
                blog_review = 0        



                
    ## 예외처리 : vscode 상태가 안좋아서 그냥 검색이 안된경우 이거나 진짜 없는 식당일 때.
    except Exception as e : 
        restaurant_name = name 
        category_name = '전처리필요'
        address = '전처리필요'
        ###''지선,지은 추가''##
        res_hours = '영업시간 없음'
        menu = '메뉴없음'
        price = "가격없음"
        visitor_review = 0
        blog_review = 0

    ## 리스트에 추가
    restaurant_name_list.append(restaurant_name)
    category_name_list.append(category_name)
    address_list.append(address)
    ###''지선, 지은 추가'' ###
    restaurant_time_list.append(res_hours)
    menu_list.append(menu)
    price_list.append(price)
    visitor_review_list.append(visitor_review)
    blog_review_list.append(blog_review)

    # # 잘돌아가나 보기 
    # print('식당 이름:',restaurant_name)
    # print('식당 카테고리:',category_name)
    # print('주소:',address)
    #     ###''지선,지은 추가''##
    # print('영업시간:', res_hours)
    # print('대표 메뉴:',menu)
    # print('가격:',price)
    # print('방문자리뷰 : ',visitor_review)
    # print('블로그리뷰 : ',blog_review)
    # print('-----------------------------')


# dataframe 만들기     
res_info_df = pd.DataFrame({'식당이름':restaurant_name_list ,
                            '업태구분' :category_name_list ,
                            '주소' :address_list ,
                            '영업시간' : restaurant_time_list,
                            '메뉴': menu_list,
                            '가격':price_list,
                            '방문자리뷰':visitor_review_list,
                            '블로그리뷰':blog_review_list,
                            })
driver.quit()

## 결과 데이터에 검색어 칼럼 추가
## 추후에 인덱스로 쓸 예정이기 떄문
res_info_df['검색어'] = res_df['검색어']


# # 엑셀로 저장 -> 나중에 하나로 합칩니다. 
res_info_df.to_excel(f'./data/restaurant_info_data(2)/restaurant_info_data/restaurant_tag_df_{start_num}_{end_num}.xlsx', index=False)
print('저장이 완료 되었습니다.')

진행 중: 100%|██████████| 98/98 [13:54<00:00,  8.52s/it]


저장이 완료 되었습니다.
